In [ ]:
import os
from pathlib import Path
import shutil
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import random


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from IPython.display import clear_output

In [ ]:
from source.visual_genome_meta_data import read_json_to_dict
from source.visual_genome_meta_data import get_image_meta_data
from source.visual_genome_meta_data import count_occurrences
from source.visual_genome_data import get_file_by_id
from source.visual_genome_to_yolo import create_class_mapping_from_list
from source.visual_genome_to_yolo import save_class_map_to_yaml
from source.visual_genome_to_yolo import convert_single_image_to_yolo
from source.visual_genome_to_yolo import read_yaml_to_class_map
from source.visual_genome_to_yolo import read_yolo_metadata
from source.visual_genome_to_yolo import visual_genome_to_yolo_data_n
from source.visual_genome_meta_data import plot_image_with_multiple_bboxes
from source.visual_genome_meta_data import get_image_ids
from source.yolo_training_structure import distribute_train_val_files as dist_train_val


In [ ]:
def clear_yolo_metadata_by_id(data_path, identifier, id_end=True):
   """
   Clear YOLO metadata file by identifier - makes it empty (removes all bounding boxes)
   
   Args:
       data_path: Path to directory containing .txt files
       identifier: Integer identifier to search for
       id_end: If True (default), select file with ID at end only.
               If False, select file with ID surrounded by underscores.
   """
   #import os
   #
   #def get_file_by_id(data_path, identifier, file_extension):
   #    filenames = []
   #    identifier_underlines = '_' + str(identifier) + '_'
   #    identifier_end = '_' + str(identifier) + '.'
   #    for file in os.listdir(str(data_path)):
   #        id_underline_bool = identifier_underlines in file
   #        id_end_bool = identifier_end in file
   #        id_bool = id_underline_bool or id_end_bool
   #        file_ext_bool = file.endswith(file_extension)
   #        if id_bool and file_ext_bool:
   #            filenames.append(file)                 
   #    return filenames
   
   # Get all matching files
   txt_files = get_file_by_id(data_path, identifier, '.txt')
   
   if not txt_files:
       print(f"No .txt file found with identifier {identifier}")
       return False
   
   # Filter based on id_end parameter
   if id_end:
       # Select only files where ID is at the end
       filtered_files = [f for f in txt_files if f.endswith(f'_{identifier}.txt')]
   else:
       # Select only files where ID is surrounded by underscores
       filtered_files = [f for f in txt_files if f'_{identifier}_' in f]
   
   if not filtered_files:
       pattern_type = "at end" if id_end else "with underscores"
       print(f"No .txt file found with identifier {identifier} {pattern_type}")
       return False
   
   # Overwrite with empty content
   file_path = os.path.join(data_path, filtered_files[0])
   with open(file_path, 'w') as f:
       pass
   
   print(f"Cleared metadata file: {filtered_files[0]}")
   return True

In [ ]:
def delete_files_by_id(data_path, identifier, file_extension, id_end=True):
   """
   Delete files by identifier.
   
   Args:
       data_path: Path to directory containing files
       identifier: Integer identifier to search for
       file_extension: File extension (e.g., '.txt', '.jpg')
       id_end: If True (default), delete only files with ID at end.
               If False, delete files with ID surrounded by underscores.
   """
   #import os
   #
   #def get_file_by_id(data_path, identifier, file_extension):
   #    filenames = []
   #    identifier_underlines = '_' + str(identifier) + '_'
   #    identifier_end = '_' + str(identifier) + '.'
   #    for file in os.listdir(str(data_path)):
   #        id_underline_bool = identifier_underlines in file
   #        id_end_bool = identifier_end in file
   #        id_bool = id_underline_bool or id_end_bool
   #        file_ext_bool = file.endswith(file_extension)
   #        if id_bool and file_ext_bool:
   #            filenames.append(file)
   #    return filenames
   
   # Get all matching files
   files = get_file_by_id(data_path, identifier, file_extension)
   
   # Filter based on id_end parameter
   if id_end:
       filtered_files = [f for f in files if f.endswith(f'_{identifier}{file_extension}')]
   else:
       filtered_files = [f for f in files if f'_{identifier}_' in f]
   
   # Delete filtered files
   deleted_count = 0
   for filename in filtered_files:
       file_path = os.path.join(data_path, filename)
       try:
           os.remove(file_path)
           print(f"Deleted: {filename}")
           deleted_count += 1
       except OSError as e:
           print(f"Error deleting {filename}: {e}")
   
   print(f"Deleted {deleted_count} files")
   return deleted_count

# Usage:
# delete_files_by_id('/path/to/files', 2324505, '.txt')  # Delete files ending with _2324505.txt

In [ ]:
def plot_image_with_yolo_grid(image_path, grid_divisions=10, figsize=(12, 10), label_size=12, 
                            zoom_region=None):
   """
   Plot image with YOLO-compatible grid overlay and labeled axes.
   
   Args:
       image_path: Path to image file (.jpg, .tif, etc.)
       grid_divisions: Number of grid divisions (default 10 for 0.1 increments)
       figsize: Figure size as (width, height) tuple (default (12, 10))
       label_size: Font size for grid labels (default 12)
       zoom_region: Tuple (x_min, y_min, x_max, y_max) in YOLO coordinates (0.0-1.0)
                   to zoom into specific region. None for full image (default)
   """
   import matplotlib.pyplot as plt
   import numpy as np
   from PIL import Image
   print(f"Current image: {image_path}")    


   
   # Load image
   img = Image.open(image_path)
   
   # Create plot with custom figure size
   fig, ax = plt.subplots(figsize=figsize)
   ax.imshow(img)
   
   # Get image dimensions
   width, height = img.size
   
   # Determine zoom bounds
   if zoom_region is None:
       x_min, y_min, x_max, y_max = 0.0, 0.0, 1.0, 1.0
   else:
       x_min, y_min, x_max, y_max = zoom_region
   
   # Create grid lines within zoom region
   grid_positions_x = np.linspace(x_min, x_max, grid_divisions + 1)
   grid_positions_y = np.linspace(y_min, y_max, grid_divisions + 1)
   
   # Vertical grid lines (x-coordinates)
   for pos in grid_positions_x:
       x_pixel = pos * width
       ax.axvline(x=x_pixel, color='red', alpha=0.7, linewidth=1)
   
   # Horizontal grid lines (y-coordinates) 
   for pos in grid_positions_y:
       y_pixel = pos * height
       ax.axhline(y=y_pixel, color='red', alpha=0.7, linewidth=1)
   
   # Create labels with appropriate precision based on grid fineness
   if grid_divisions <= 10:
       decimal_places = 1
   elif grid_divisions <= 100:
       decimal_places = 2
   else:
       decimal_places = 3
   
   x_labels = [f"{pos:.{decimal_places}f}" for pos in grid_positions_x]
   y_labels = [f"{pos:.{decimal_places}f}" for pos in grid_positions_y]
   
   # Set tick positions and labels with custom font size
   ax.set_xticks([pos * width for pos in grid_positions_x])
   ax.set_xticklabels(x_labels, fontsize=label_size)
   ax.set_yticks([pos * height for pos in grid_positions_y])
   ax.set_yticklabels(y_labels, fontsize=label_size)
   
   # Set zoom limits if specified
   if zoom_region is not None:
       ax.set_xlim(x_min * width, x_max * width)
       ax.set_ylim(y_max * height, y_min * height)  # Note: y-axis is flipped in images
   
   # Labels with custom font size
   ax.set_xlabel('YOLO X-coordinate (normalized)', fontsize=label_size)
   ax.set_ylabel('YOLO Y-coordinate (normalized)', fontsize=label_size)
   
   zoom_text = f" (zoomed: {x_min:.2f}-{x_max:.2f}, {y_min:.2f}-{y_max:.2f})" if zoom_region else ""
   ax.set_title(f'YOLO Grid Overlay - {image_path}{zoom_text}', fontsize=label_size)
   
   plt.tight_layout()
   plt.show()

In [ ]:
def parse_zoom_input(user_input):
   """Parse zoom region from user input"""
   if not user_input.strip():
       return None
   
   try:
       coords = [float(x.strip()) for x in user_input.strip().split(',')]
       if len(coords) == 4:
           return tuple(coords)
       else:
           print("Invalid format. Please enter 4 values: x_min, y_min, x_max, y_max")
           return False
   except ValueError:
       print("Invalid input. Please enter numbers separated by commas.")
       return False

In [ ]:
def grid_coords_to_yolo(top_left, bottom_right, image_path):
   """
   Convert grid coordinates to YOLO format bounding box.
   
   Args:
       top_left: Tuple (x, y) of top-left corner in grid coordinates (0.0-1.0)
       bottom_right: Tuple (x, y) of bottom-right corner in grid coordinates (0.0-1.0)
       image_path: Path to image (needed to get dimensions for validation)
   
   Returns:
       Tuple (x_center, y_center, width, height) in YOLO format (all 0.0-1.0)
   """
   from PIL import Image
   
   # Load image to validate (optional, but good practice)
   img = Image.open(image_path)
   img_width, img_height = img.size
   
   # Extract coordinates
   x1, y1 = top_left      # Top-left corner
   x2, y2 = bottom_right  # Bottom-right corner
   
   # Validate coordinates are in correct order
   if x1 >= x2 or y1 >= y2:
       raise ValueError(f"Invalid coordinates: top_left {top_left} should be above and left of bottom_right {bottom_right}")
   
   # Validate coordinates are in valid range
   if not (0 <= x1 <= 1 and 0 <= y1 <= 1 and 0 <= x2 <= 1 and 0 <= y2 <= 1):
       raise ValueError("All coordinates must be between 0.0 and 1.0")
   
   # Calculate YOLO format
   x_center = (x1 + x2) / 2
   y_center = (y1 + y2) / 2
   width = x2 - x1
   height = y2 - y1
   
   print(f"Grid coordinates: top_left=({x1:.3f}, {y1:.3f}), bottom_right=({x2:.3f}, {y2:.3f})")
   print(f"YOLO format: center=({x_center:.6f}, {y_center:.6f}), size=({width:.6f}, {height:.6f})")
   
   return x_center, y_center, width, height

# Usage examples:
# yolo_coords = grid_coords_to_yolo((0.2, 0.1), (0.8, 0.7), 'image.jpg')
# x_center, y_center, width, height = grid_coords_to_yolo((0.3, 0.2), (0.9, 0.6), 'image.jpg')

In [ ]:
def create_yolo_metadata_file(output_path, bounding_boxes):
    """
    Create and save YOLO metadata file with multiple bounding boxes.
    
    Args:
        output_path: Path where to save the .txt file
        bounding_boxes: List of tuples, each containing (class_id, x_center, y_center, width, height)
    """
    with open(output_path, 'w') as f:
        for class_id, x_center, y_center, width, height in bounding_boxes:
            yolo_line = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
            f.write(yolo_line)
    
    print(f"YOLO metadata saved to: {output_path}")
    print(f"Added {len(bounding_boxes)} bounding boxes")



### Define paths: 

In [ ]:
#root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project')
#root_path = Path('/Users/stephanehess/Documents/CAS_AML/dias_digit_project/test_yolo_object_train')

project_path = Path.cwd()
root_path = (project_path / '..').resolve()
#root_path = (project_path / '..' / 'test_yolo_object_train').resolve()

data_path = root_path / 'visual_genome_data'
#data_path = root_path / 'visual_genome_data_all'
yolo_path = root_path / 'visual_genome_yolo'
#yolo_path = root_path / 'visual_genome_yolo_all'

In [ ]:
print(data_path)
print(yolo_path)

In [ ]:
data_path

In [ ]:
yolo_path

In [ ]:
os.listdir(yolo_path)

### Read in objects file with meta data about visual genome data: 

In [ ]:
objects_file_path = data_path/'objects.json'


In [ ]:
objects = read_json_to_dict(objects_file_path)

### Get image identifiers: 

In [ ]:
image_id_list = get_image_ids(data_path)
image_id_list.sort()
len(image_id_list)

In [ ]:
image_id_list[0:3]

In [ ]:
len(objects)

### Choose the desired objects:

In [ ]:
#desired_objects = ['forest', 'mountain', 'mountains', 'building', 'house', 
#                   'church', 'city', 'village', 'lake', 'river', 'stream', 'glacier']

#desired_objects = ['mountain']
desired_objects = ['church']
#desired_objects = ['lighthouse']


desired_objects

### Create class map based on desired objects: 

In [ ]:
class_map = create_class_mapping_from_list(desired_objects)

In [ ]:
class_map

In [ ]:

file_path = str(yolo_path) + '/'

output_path = file_path + 'class_map.yaml'
save_class_map_to_yaml(class_map, output_path)

### Check content of class_map.yaml file:

In [ ]:
file_list = os.listdir(yolo_path)
for filename in file_list:
    if filename.split('_')[-1] == 'map.yaml':
        yaml_file_name = filename
yaml_file_name

In [ ]:
yaml_path = yolo_path/yaml_file_name

class_map = read_yaml_to_class_map(str(yaml_path))

# Print the class mapping
print(class_map)

In [ ]:
len(image_id_list)

In [ ]:
#convert_single_image_to_yolo(objects[0], class_map, data_path, yolo_path)

### Create yolo compatible meta data files (bounding box information) for images containing the desired object:

In [ ]:

objects_and_ids = (objects, desired_objects, image_id_list)
paths = (data_path, yolo_path)

label_paths_w, occurrence_counts = visual_genome_to_yolo_data_n(objects_and_ids, paths, class_map)
len(label_paths_w)

In [ ]:
occurrence_counts

In [ ]:
label_paths_w[0:3]

In [ ]:
class_map

In [ ]:
label_paths_w[0:3]

### Get number of required images without desired object for balanced data set:

In [ ]:
desired_objects

In [ ]:
desired_objects[0]

In [ ]:
if len(desired_objects) == 1:
    number_occurrences = occurrence_counts[desired_objects[0]]
    print(number_occurrences)
else:
    number_occurrences = 'No unique answer: more than one desired objects!'
    print(number_occurrences)

In [ ]:
number_images_without = round((number_occurrences/100) * 30)
number_images_without

### Create meta data text files for images without desired object:

In [ ]:
label_paths_n, occurrence_counts = visual_genome_to_yolo_data_n(objects_and_ids, paths, class_map,
                                                           with_class = False, number_of_images = number_images_without)
len(label_paths_n)

In [ ]:
label_paths_n[0:3]

### The paths to the meta data files contain the image ids defining the image files to be used

#### Paths to label files with desired objects:

In [ ]:
# Paths to label files with desired objects:
print(type(label_paths_w))
print(len(label_paths_w))

#### Paths to label files without desired objects:

In [ ]:
# Paths to label files without desired objects: 
print(type(label_paths_n))
print(len(label_paths_n))

In [ ]:
label_paths = label_paths_w + label_paths_n
len(label_paths)

### Plot images containing desired object and use class_map file to plot bounding boxes:

In [ ]:
for label_path in label_paths[0:3]:

    img_id = label_path.split('_')[-1].split('.')[0]
    print(img_id)
    
    labels, bboxes = read_yolo_metadata(label_path, class_map)
    class_names = list(labels)
    image_path_gen = data_path/'visual_genome_'
    image_path = str(image_path_gen) + str(img_id) + '.jpg'
    print(image_path)
    plot_image_with_multiple_bboxes(image_path, bboxes, class_names)
    

In [ ]:
files_to_delete = []
files_to_clear = []
files_to_annotate = []

In [ ]:
# Files with incomplete annotation need to be annotated
# manually: 
files_to_annotate_incomplete = ['visual_genome_2569.jpg',
'visual_genome_2861.jpg',
'visual_genome_3443.jpg',
'visual_genome_1592640.jpg',
'visual_genome_2414118.jpg',
'visual_genome_2407261.jpg',
'visual_genome_2404517.jpg',
'visual_genome_2403320.jpg',
'visual_genome_2401281.jpg',
'visual_genome_2395319.jpg',
'visual_genome_2393416.jpg',
'visual_genome_2391458.jpg',
'visual_genome_2390174.jpg',
'visual_genome_2387016.jpg',
'visual_genome_2386301.jpg',
'visual_genome_2381437.jpg',
'visual_genome_2380423.jpg',
'visual_genome_2377936.jpg',
'visual_genome_2368816.jpg',
'visual_genome_2365476.jpg',
'visual_genome_2362585.jpg',
'visual_genome_2350086.jpg',
'visual_genome_2347182.jpg',
'visual_genome_2338872.jpg',
'visual_genome_2338000.jpg',
'visual_genome_2335182.jpg',
'visual_genome_2331993.jpg',
'visual_genome_2322632.jpg',
'visual_genome_2412891.jpg',
'visual_genome_2411961.jpg']

files_to_annotate.extend(files_to_annotate_incomplete)

In [ ]:
# Images showing the interior of Church with architectural characteristics,
# don't know what to do with them so delete the respective meta-data-files: 

files_interior_with_features = ['visual_genome_4164.jpg',
'visual_genome_4166.jpg',
'visual_genome_498224.jpg',
'visual_genome_2414880.jpg',
'visual_genome_2410675.jpg',
'visual_genome_2397641.jpg',
'visual_genome_2376269.jpg',
'visual_genome_2374462.jpg',
'visual_genome_2373973.jpg',
'visual_genome_2368090.jpg',
'visual_genome_2360455.jpg',
'visual_genome_2350745.jpg',
'visual_genome_2341636.jpg',
'visual_genome_2327026.jpg',
'visual_genome_2322689.jpg',
'visual_genome_2417527.jpg']

files_to_delete_interior = ['visual_genome_4164.txt',
'visual_genome_4166.txt',
'visual_genome_498224.txt',
'visual_genome_2414880.txt',
'visual_genome_2410675.txt',
'visual_genome_2397641.txt',
'visual_genome_2376269.txt',
'visual_genome_2374462.txt',
'visual_genome_2373973.txt',
'visual_genome_2368090.txt',
'visual_genome_2360455.txt',
'visual_genome_2350745.txt',
'visual_genome_2341636.txt',
'visual_genome_2327026.txt',
'visual_genome_2322689.txt',
'visual_genome_2417527.txt']

files_to_delete.extend(files_to_delete_interior)

In [ ]:
# Images showing interior of Church without features
# can be considered as images not showing churches
# i.e. content should be cleared from meta data files: 
files_interior_without_features = ['visual_genome_2404416.jpg',
'visual_genome_2396536.jpg',
'visual_genome_2389289.jpg',
'visual_genome_2384205.jpg',
'visual_genome_2352008.jpg',
'visual_genome_2416136.jpg']

files_to_clear_interior =['visual_genome_2404416.txt',
'visual_genome_2396536.txt',
'visual_genome_2389289.txt',
'visual_genome_2384205.txt',
'visual_genome_2352008.txt',
'visual_genome_2416136.txt']

files_to_clear.extend(files_to_clear_interior)

In [ ]:
# Written word "church":
# A character string signifying church is not a church, 
# content must be cleared from meta data files:
files_word_church = ['visual_genome_61591.jpg',
'visual_genome_1591985.jpg',
'visual_genome_2386541.jpg',
'visual_genome_2379525.jpg',
'visual_genome_2363367.jpg']

files_to_clear_word = ['visual_genome_61591.txt',
'visual_genome_1591985.txt',
'visual_genome_2386541.txt',
'visual_genome_2379525.txt',
'visual_genome_2363367.txt']

files_to_clear.extend(files_to_clear_word)

In [ ]:
# Church like buildings will be removed:

#Church like building without annotation:
files_church_like_no_bb = ['visual_genome_1592399.jpg',
'visual_genome_2407154.jpg',
'visual_genome_2353987.jpg',
'visual_genome_2324725.jpg',
'visual_genome_2415943.jpg',
'visual_genome_54.jpg']

#Church like building with annotation: 
files_church_like_with_bb = ['visual_genome_2368326.jpg',
'visual_genome_2411961.jpg']

#Church like building without annotation:
files_to_delete_church_like_no_bb = ['visual_genome_1592399.txt',
'visual_genome_2407154.txt',
'visual_genome_2353987.txt',
'visual_genome_2324725.txt',
'visual_genome_2415943.txt',
'visual_genome_54.txt']

#Church like building with annotation: 
files_to_delete_church_like_with_bb = ['visual_genome_2368326.txt',
'visual_genome_2411961.txt']

files_to_clear.extend(files_to_delete_church_like_no_bb)
files_to_clear.extend(files_to_delete_church_like_with_bb)

In [ ]:
# Church not visible: 
# Images where no church is visible or recognisable should
# be considered as not showing a church: There meta data
# files should be cleared (content removed).

files_church_not_visible = ['visual_genome_2389082.jpg',
'visual_genome_2387130.jpg',
'visual_genome_2373067.jpg',
'visual_genome_2369703.jpg',
'visual_genome_2369464.jpg',
'visual_genome_2366835.jpg',
'visual_genome_2354921.jpg',
'visual_genome_2332124.jpg',
'visual_genome_2319489.jpg',
'visual_genome_2412891.jpg']

files_to_clear_not_visible = ['visual_genome_2389082.txt',
'visual_genome_2387130.txt',
'visual_genome_2373067.txt',
'visual_genome_2369703.txt',
'visual_genome_2369464.txt',
'visual_genome_2366835.txt',
'visual_genome_2354921.txt',
'visual_genome_2332124.txt',
'visual_genome_2319489.txt',
'visual_genome_2412891.txt']

files_to_clear.extend(files_to_clear_not_visible)

In [ ]:
print(files_to_delete[0:3])
print(files_to_clear[0:3])
print(files_to_annotate[0:3])

### Remove meta data files of images not to include in the analysis: 

#### Check if files to delete are there:

In [ ]:
files_found = []
for file_to_delete in files_to_delete:
    end_part = file_to_delete.split('_')[-1].split('.')[0]
    print(end_part)
    file_found = get_file_by_id(yolo_path, int(end_part), 'txt')[0]
    files_found.append(file_found)
files_found

#### Delete files: 

In [ ]:
for file_to_delete in files_to_delete:
    end_part = file_to_delete.split('_')[-1].split('.')[0]
    print(end_part)
    delete_files_by_id(yolo_path, int(end_part), '.txt')

#### Check if files to delete are still there (they should be gone):

In [ ]:
#del file_found
files_found = []
for file_to_delete in files_to_delete:
    end_part = file_to_delete.split('_')[-1].split('.')[0]
    print(end_part)
    files_found = get_file_by_id(yolo_path, int(end_part), 'txt')
    if len(files_found) > 0:
        file_found = files_found[0]
        print(file_found)
        files_found.append(file_found)
    else:
        files_found.extend(files_found)
files_found

#### Check content of files to be cleared: 

In [ ]:
for file in files_to_clear[0:7]:
    file_path = os.path.join(yolo_path, file)
    print(file_path)
    with open(file_path, 'r') as file:
        content = file.read()
        print(content)

#### Clear files:

In [ ]:
for file_to_clear in files_to_clear:
    end_part = file_to_clear.split('_')[-1].split('.')[0]
    
    clear_yolo_metadata_by_id(yolo_path, int(end_part), id_end=True)

#### Check if content was cleared from files:

In [ ]:
for file in files_to_clear[0:7]:
    file_path = os.path.join(yolo_path, file)
    print(file_path)
    with open(file_path, 'r') as file:
        content = file.read()
        print(content)

In [ ]:
def parse_zoom_input(user_input):
   """Parse zoom region from user input"""
   if not user_input.strip():
       return None
   
   try:
       coords = [float(x.strip()) for x in user_input.strip().split(',')]
       if len(coords) == 4:
           return tuple(coords)
       else:
           print("Invalid format. Please enter 4 values: x_min, y_min, x_max, y_max")
           return False
   except ValueError:
       print("Invalid input. Please enter numbers separated by commas.")
       return False



In [ ]:
for file in files_to_annotate[0:15]:

    img_id = file.split('_')[-1].split('.')[0]
    print(img_id)

    label_file = 'visual_genome_' + img_id + '.txt'
    label_path = os.path.join(yolo_path, label_file)
    print(label_path)
    
    
    labels, bboxes = read_yolo_metadata(label_path, class_map)
    class_names = list(labels)
    
    image_path = os.path.join(data_path, file)
    
    print(image_path)
    
    plot_image_with_multiple_bboxes(image_path, bboxes, class_names)

In [ ]:
files_to_annotate_selection = files_to_annotate[0:7]

img_idx = 0
exit_viewer = False

while img_idx < len(files_to_annotate_selection) and not exit_viewer:
    img_file = files_to_annotate_selection[img_idx] 
    image_path = os.path.join(data_path, img_file)
    zoom_region = None

    
    while True:

        plot_image_with_yolo_grid(image_path, grid_divisions=15, figsize=(20,16), label_size=22, zoom_region=zoom_region)
        plt.show()
        
        user_input = input("Press Enter for next image, enter zoom region (x_min,y_min,x_max,y_max), or 'q' to quit: ")
        plt.close()
        clear_output(wait=True)
        
        if user_input.strip().lower() in ['q', 'quit']:
            exit_viewer = True
            break
        elif not user_input.strip():
            break
        else:
            parsed_zoom = parse_zoom_input(user_input)
            if parsed_zoom is None:
                break
            elif parsed_zoom is False:
                continue
                
            else:
                zoom_region = parsed_zoom
        ####
        # Auto-adjust grid divisions based on zoom region size
        if zoom_region is not None:
            x_min, y_min, x_max, y_max = zoom_region
            zoom_width = x_max - x_min
            zoom_height = y_max - y_min
            zoom_area = zoom_width * zoom_height
            
            # Scale grid divisions inversely with zoom area (smaller area = more divisions)
            if zoom_area < 0.1:
                grid_divisions = 50
            elif zoom_area < 0.25:
                grid_divisions = 30
            elif zoom_area < 0.5:
                grid_divisions = 20
            else:
                grid_divisions = 10
        else:
            grid_divisions = 20
            
    ####
    
    img_idx += 1

In [ ]:
image_path

In [ ]:
image_files_no_object = [

]

In [ ]:
files_for_annotation = [
{'image_file': 'visual_genome_2569.jpg', 
 'objects': [
     {'object_class': 0, 
      'top_left_values': [(0.33, 0.42)],
      'bottom_right_values': [(0.56, 0.58)]}
 ]
 },
{'image_file': 'visual_genome_2861.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.4, 0.01)], 
      'bottom_right_values': [(0.87, 0.6)]}
 ]
 },
{'image_file': 'visual_genome_3443.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.18, 0.03)], 
      'bottom_right_values': [(0.47, 0.27)]}
 ]
 },
{'image_file': 'visual_genome_1592640.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.0, 0.02)], 
      'bottom_right_values': [(0.6, 0.67)]}
 ]
 },
{'image_file': 'visual_genome_2414118.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.14, 0.01)], 
      'bottom_right_values': [(0.99, 0.99)]}
 ]
 },
{'image_file': 'visual_genome_2407261.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.27, 0.03)], 
      'bottom_right_values': [(0.6, 0.87)]}
 ]
 },
{'image_file': 'visual_genome_2404517.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.01, 0.01)], 
      'bottom_right_values': [(0.99, 0.99)]}
 ]
 },
{'image_file': 'visual_genome_2403320.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.0, 0.0)], 
      'bottom_right_values': [(0.99, 0.99)]}
 ]
 },
  {'image_file': 'visual_genome_2401281.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.0, 0.03)], 
      'bottom_right_values': [(0.9, 0.9)]}
 ]
 },
{'image_file': 'visual_genome_2395319.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.0, 0.0)], 
      'bottom_right_values': [(0.99, 0.99)]}
 ]
 },
{'image_file': 'visual_genome_2393416.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.0, 0.0)], 
      'bottom_right_values': [(0.99, 0.99)]}
 ]
 },
{'image_file': 'visual_genome_2391458.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.47, 0.27)], 
      'bottom_right_values': [(0.8, 0.9)]}
 ]
 },
{'image_file': 'visual_genome_2390174.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.0, 0.0)], 
      'bottom_right_values': [(0.99, 0.99)]}
 ]
 },
{'image_file': 'visual_genome_2387016.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.0, 0.0)], 
      'bottom_right_values': [(0.99, 0.99)]}
 ]
 },
{'image_file': 'visual_genome_2386301.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.04, 0.07)], 
      'bottom_right_values': [(0.25, 0.34)]}
 ]
 },
{'image_file': 'visual_genome_2381437.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.22, 0.02)], 
      'bottom_right_values': [(0.9, 0.82)]}
 ]
 },
{'image_file': 'visual_genome_2380423.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.0, 0.0)], 
      'bottom_right_values': [(0.93, 0.99)]}
 ]
 },
{'image_file': 'visual_genome_2377936.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.73, 0.07)], 
      'bottom_right_values': [(0.99, 0.27)]}
 ]
 },
{'image_file': 'visual_genome_2368816.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.23, 0.03)], 
      'bottom_right_values': [(0.88, 0.97)]}
 ]
 },
{'image_file': 'visual_genome_2365476.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.44, 0.11), (0.26, 0.13)], 
      'bottom_right_values': [(0.73, 0.38), (0.9, 0.46)]}
 ]
 },
{'image_file': 'visual_genome_2362585.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.0, 0.0)], 
      'bottom_right_values': [(0.99, 0.99)]}
 ]
 },
{'image_file': 'visual_genome_2350086.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.4, 0.0)], 
      'bottom_right_values': [(0.99, 0.99)]}
 ]
 },
{'image_file': 'visual_genome_2347182.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.07, 0.13)], 
      'bottom_right_values': [(0.99, 0.99)]}
 ]
 },
{'image_file': 'visual_genome_2338872.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.27, 0.0)], 
      'bottom_right_values': [(0.99, 0.6)]}
 ]
 },
{'image_file': 'visual_genome_2338000.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.0, 0.0)], 
      'bottom_right_values': [(0.99, 0.99)]}
 ]
 },
{'image_file': 'visual_genome_2335182.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.35, 0.2)], 
      'bottom_right_values': [(0.81, 0.67)]}
 ]
 },
{'image_file': 'visual_genome_2331993.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.13, 0.13), (0.6, 0.33)], 
      'bottom_right_values': [(0.47, 0.67), (0.93, 0.55)]}
 ]
 },
{'image_file': 'visual_genome_2322632.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.27, 0.02)], 
      'bottom_right_values': [(0.93, 0.73)]}
 ]
 },
{'image_file': 'visual_genome_2412891.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.13, 0.11)], 
      'bottom_right_values': [(0.95, 0.95)]}
 ]
 },
{'image_file': 'visual_genome_2411961.jpg', 
 'objects': [
     {'object_class': 0,
      'top_left_values': [(0.27, 0.12)], 
      'bottom_right_values': [(0.99, 0.87)]}
 ]
 }
]

In [ ]:
meta_data_for_annotation = []
file_names_for_annotation = []

for file in files_for_annotation:
    file_name = file['image_file']
    #print(file_name)
    file_names_for_annotation.append(file_name)
    image_path = os.path.join(data_path, file_name)
    img_id = image_path.split('_')[-1].split('.')[0]
    #print(img_id)
    label_file = 'visual_genome_' + img_id + '.txt'
    label_path = os.path.join(yolo_path, label_file)
    #print(label_path)
    file_meta_data = {'file_name': file_name,
                     'output_path': label_path,
                     'objects': []}
    
    for img_object in file['objects']:
        
        object_class = img_object['object_class']
        #print(object_class)
        
        top_left_values = img_object['top_left_values']
        #print(top_left_values)
        
        bottom_right_values = img_object['bottom_right_values']
        #print(bottom_right_values)
        
        #object_meta_data = []
        for top_left, bottom_right in zip(top_left_values, bottom_right_values):
            x_center, y_center, width, height = grid_coords_to_yolo(top_left, bottom_right, image_path)
            yolo_meta_data = (object_class, x_center, y_center, width, height)
            #object_meta_data.append(yolo_meta_data)
            file_meta_data['objects'].append(yolo_meta_data)

    meta_data_for_annotation.append(file_meta_data)
        
        
        

In [ ]:
files_newly_annotated = file_names_for_annotation + image_files_no_object

In [ ]:
for file_meta_data in meta_data_for_annotation:
    #print(file_meta_data)
    label_path = file_meta_data['output_path']
    yolo_meta_data = file_meta_data['objects']
    #print(type(yolo_meta_data))
    create_yolo_metadata_file(label_path, yolo_meta_data)

In [ ]:
# Method 3: For multiple empty annotation files
#image_names = ['image_001.jpg', 'image_002.jpg', 'image_003.jpg']
for img_name in image_files_no_object:
    #print(img_name)
    txt_name = img_name.replace('.jpg', '.txt')
    #print(txt_name)
    output_path = os.path.join(yolo_path, txt_name)
    with open(output_path, 'w') as file:
        pass


In [ ]:


for file in files_newly_annotated:

    img_id = file.split('_')[-1].split('.')[0]
    print(img_id)

    label_file = 'visual_genome_' + img_id + '.txt'
    label_path = os.path.join(yolo_path, label_file)
    print(label_path)
    
    
    labels, bboxes = read_yolo_metadata(label_path, class_map)
    class_names = list(labels)
    
    #image_path = os.path.join(data_path, file)
    image_path = data_path / file
    
    print(image_path)
    
    plot_image_with_multiple_bboxes(image_path, bboxes, class_names)

### Since some meta data files have been deleted, get label paths list again:

In [ ]:
label_paths = []
for file_name in os.listdir(yolo_path): 
    if file_name.split('.')[-1] == 'txt':
        label_path = os.path.join(yolo_path, file_name)
        label_paths.append(label_path)

len(label_paths)
        

In [ ]:
label_paths[0:7]

In [ ]:
root_path

### Create file structure to train for recognition of desired object class:

In [ ]:
train_val_trial_path = root_path / 'yolo_object_train'


if not os.path.exists(train_val_trial_path):
    os.makedirs(train_val_trial_path)

In [ ]:
train_val_trial_path

In [ ]:
train_data_path = train_val_trial_path / 'train'

if not os.path.exists(train_data_path):
    os.makedirs(train_data_path)


In [ ]:
os.getcwd()

In [ ]:
train_imgages_path = train_data_path / 'images'

if not os.path.exists(train_imgages_path):
    os.makedirs(train_imgages_path)

train_labels_path = train_data_path / 'labels'

if not os.path.exists(train_labels_path):
    os.makedirs(train_labels_path)


In [ ]:
val_data_path = train_val_trial_path  / 'val'
if not os.path.exists(val_data_path):
    os.makedirs(val_data_path)

os.listdir(val_data_path)

In [ ]:
val_imgages_path = val_data_path / 'images'
if not os.path.exists(val_imgages_path):
    os.makedirs(val_imgages_path)

val_labels_path = val_data_path / 'labels'
if not os.path.exists(val_labels_path):
    os.makedirs(val_labels_path)


In [ ]:
train_images_path = train_data_path / 'images'
if not os.path.exists(train_images_path):
    os.makedirs(train_images_path)

train_labels_path = train_data_path / 'labels'
if not os.path.exists(train_labels_path):
    os.makedirs(train_labels_path)


In [ ]:
val_images_path = val_data_path / 'images'
if not os.path.exists(val_images_path):
    os.makedirs(val_images_path)

val_labels_path = val_data_path / 'labels'
if not os.path.exists(val_labels_path):
    os.makedirs(val_labels_path)


In [ ]:
train_images_grey_path = train_val_trial_path / 'train_grey/images'
 
if not os.path.exists(train_images_grey_path):
    os.makedirs(train_images_grey_path)

In [ ]:
val_images_grey_path = train_val_trial_path / 'val_grey/images'

if not os.path.exists(val_images_grey_path):
    os.makedirs(val_images_grey_path)

In [ ]:
train_val_trial_path

In [ ]:
val_data_path

### Make a list of all selected image ids by looping through the label paths: 

In [ ]:
round_counter = 0
selected_image_ids = []
for label_path in label_paths:
    #print(label_path)
    last_part = label_path.split('_')[-1]
    image_id = int(last_part.split('.')[0])
    selected_image_ids.append(image_id)
    round_counter += 1
    #if round_counter > 2:
     #   break

In [ ]:
print(len(selected_image_ids))
print(selected_image_ids[0:7])


### Shuffle selected image ids and subdivide them into training and validation set:

In [ ]:
import random

def split_shuffle(string_list, split_ratio=0.8):
    # Shuffle the list in place
    random.shuffle(string_list)
    
    # Calculate split point
    split_point = int(len(string_list) * split_ratio)
    
    # Split the list
    train_set = string_list[:split_point]
    test_set = string_list[split_point:]
    
    return train_set, test_set

In [ ]:
print(len(selected_image_ids))
train_ids, val_ids = split_shuffle(selected_image_ids)
print(len(train_ids))
print(len(val_ids))

### Get a list of all image files:

In [ ]:
all_file_list = os.listdir(data_path)
image_file_list = []
for filename in all_file_list:
    file_extension = filename.split('.')[-1]
    if file_extension == 'jpg':
        image_file_list.append(filename)
    
print(len(image_file_list))

image_file_list[0:7]

### Loop through image file list and label_paths list and move files to the training or validation folder according to their id:

In [ ]:
train_imgages_path

In [ ]:
train_labels_path

In [ ]:
train_ids[0:7]

In [ ]:
val_ids[0:7]

In [ ]:
image_file_list.sort(key=len, reverse=True)
for file in image_file_list:
    print(len(file))
    print(file)
    break

In [ ]:
label_paths[0:2]

In [ ]:
import shutil

In [ ]:
dist_train_val(image_file_list, train_ids, val_ids, data_path, 
               train_imgages_path, val_imgages_path)
dist_train_val(label_paths, train_ids, val_ids, yolo_path, 
                           train_labels_path, val_labels_path, full_path=True)


### Convert images to grey scale images

In [ ]:
import cv2
import os
from pathlib import Path

def convert_dataset_to_grayscale(input_dir, output_dir):
    """Convert all images in dataset to grayscale"""
    os.makedirs(output_dir, exist_ok=True)
    
    for img_path in Path(input_dir).glob('*.jpg'):
        # Read image
        img = cv2.imread(str(img_path))
        
        # Convert to grayscale
        gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Convert back to 3-channel (YOLO expects 3 channels)
        gray_3channel = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)
        
        # Save
        output_path = os.path.join(output_dir, img_path.name)
        cv2.imwrite(output_path, gray_3channel)


In [ ]:
train_data_path 

In [ ]:
val_data_path

In [ ]:
train_data_path

In [ ]:
# Convert your training images
convert_dataset_to_grayscale(train_images_path, train_images_grey_path)
convert_dataset_to_grayscale(val_images_path, val_images_grey_path)

In [ ]:
os.getcwd()

In [ ]:
test_files_path = root_path / 'test_files'
test_files_grey_path = root_path / 'test_files_grey'

In [ ]:
convert_dataset_to_grayscale(test_files_path, test_files_grey_path)

### Adapt brightness and contrast of grey scale images to make them look old:

In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageEnhance, ImageFilter
import random
import os

In [ ]:
def simulate_specific_old_effects(image_path, output_path):
    """
    Apply specific effects that match your old photos.
    Adjust these based on what you observe in your test images.
    """
    img = Image.open(image_path).convert('RGB')
    
    # Heavy JPEG compression (very low quality)
    img.save('temp.jpg', 'JPEG', quality=15)
    img = Image.open('temp.jpg')
    os.remove('temp.jpg')
    
    # Significant brightness reduction
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(0.8)
    
    # Low contrast
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(0.9)
    
    # Add significant noise
    img_array = np.array(img)
    noise = np.random.normal(0, 0.1, img_array.shape).astype(np.uint8)
    img_array = np.clip(img_array.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    img = Image.fromarray(img_array)
    
    # Strong blur
    img = img.filter(ImageFilter.GaussianBlur(radius=1.5))
    
    img.save(output_path, 'JPEG', quality=85) 


In [ ]:
def process_training_dataset_spec(input_dir, output_dir, augmentation_ratio=0.5):
    """
    Process a directory of training images to simulate old photo effects.
    
    Args:
        input_dir: Directory with original images
        output_dir: Directory to save processed images
        augmentation_ratio: Fraction of images to augment (0.5 = 50%)
    """
    os.makedirs(output_dir, exist_ok=True)
    
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.tif'))]
    
    for img_file in image_files:
        input_path = os.path.join(input_dir, img_file)
        
        # Always copy original
        original_output = os.path.join(output_dir, img_file)
        #img = Image.open(input_path)
        #img.save(original_output)
        

        # Create filename for augmented version
        name, ext = os.path.splitext(img_file)
        aug_filename = f"{name}_aged{ext}"
        aug_output = os.path.join(output_dir, aug_filename)
        #aug_output = os.path.join(output_dir, img_file)
        
        # Apply aging effects with random intensity
        intensity = random.uniform(0.3, 0.8)
        simulate_specific_old_effects(input_path, aug_output)
    
    print(f"Processed {len(image_files)} images in {input_dir}")

In [ ]:
train_images_aged_path = train_val_trial_path / 'train_grey/images_aged'
val_images_aged_path = train_val_trial_path / 'val_grey/images_aged'

In [ ]:
process_training_dataset_spec(train_images_grey_path, train_images_aged_path)
process_training_dataset_spec(val_images_grey_path, val_images_aged_path)

### Move all grey images back to train and val folders (overwriting the coloured images)

In [ ]:
grey_image_list = os.listdir(train_images_grey_path)
for image in grey_image_list:
    grey_image_path = train_images_grey_path / image
    dest_grey_image_path = train_images_path / image
    shutil.copy(grey_image_path, dest_grey_image_path)

In [ ]:
grey_image_list = os.listdir(val_images_grey_path)
for image in grey_image_list:
    grey_image_path = val_images_grey_path / image
    dest_grey_image_path = val_images_path / image
    shutil.copy(grey_image_path, dest_grey_image_path)

In [ ]:
grey_image_list = os.listdir(test_files_grey_path)
for image in grey_image_list:
    grey_image_path = test_files_grey_path / image
    dest_grey_image_path = test_files_path / image
    shutil.copy(grey_image_path, dest_grey_image_path)

In [ ]:
from source.visual_genome_aged_effect import simulate_specific_old_effects
from source.visual_genome_aged_effect import process_training_dataset_spec
from source.visual_genome_aged_effect import copy_with_new_id
from source.visual_genome_aged_effect import add_new_id_img_meta

In [ ]:
train_images_aged_path

### Get identifiers of training and validation images:

In [ ]:
train_img_ids = get_image_ids(str(train_images_path))
val_img_ids = get_image_ids(str(val_images_path))
train_max_id = max(train_img_ids)
val_max_id = max(val_img_ids)
max_id = max([train_max_id, val_max_id])
max_id

### Create and add new identifiers to aged versions of images; move files to train and val folder:

#### The aged image versions are added to the original grey scale images, so that for every image there is an original and a grey scale version

In [ ]:
file_extensions = ['.jpg', '.txt']
tag = 'aged'
add_new_id_img_meta(train_images_aged_path, train_labels_path, 
                    train_images_path, train_labels_path, 
                    train_img_ids, max_id, tag, file_extensions)

In [ ]:
train_img_ids = get_image_ids(str(train_images_path))
val_img_ids = get_image_ids(str(val_images_path))
train_max_id = max(train_img_ids)
val_max_id = max(val_img_ids)
max_id = max([train_max_id, val_max_id])
max_id

In [ ]:
file_extensions = ['.jpg', '.txt']
tag = 'aged'

add_new_id_img_meta(val_images_aged_path, val_labels_path, 
                    val_images_path, val_labels_path, 
                    val_img_ids, max_id, tag, file_extensions)

### Copy yaml file to training folder:

In [ ]:
yaml_path

In [ ]:
yolo_yaml_path = train_val_trial_path / yaml_file_name
yolo_yaml_path

In [ ]:
shutil.copy(yaml_path, yolo_yaml_path)